# Deep Neural Networks: Libraries comparison

## 1) Numpy
L-layers neural network for binary and multi-class classification built on numpy.

### Activation functions
- **ReLU**: $A = RELU(Z) = max(0, Z)$
- **Sigmoid**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$

### Cost Function
- **Cross-Entropy Loss**: $ \frac{1}{m}\sum^m_{i=1} y^{(i)}*log(\hat y^{[L](i)}) + (1-y^{(i)})*log(1- \hat y^{[L](i)})$

prova